In [2]:
!pip3 install pymisp pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.1 MB/s eta 0:00:00


In [ ]:
from Crypto.Cipher import AES
import base64
import hashlib
from Crypto.Random import get_random_bytes

def encrypt(plaintext, key):
    key = hashlib.sha256(key.encode()).digest()
    cipher = AES.new(key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(plaintext.encode('utf-8'))
    encrypted_data = cipher.nonce + ciphertext
    return base64.b64encode(encrypted_data).decode('utf-8')

plaintext = "MISP_API_KEY"
key = "ais3_2025_cycraft"

# encrypted_text = encrypt(plaintext, key)
# print("Encrypted Text:", encrypted_text)

In [3]:
from Crypto.Cipher import AES
import base64
import hashlib

def decrypt(encrypted_text, key):
    key = hashlib.sha256(key.encode()).digest()
    encrypted_text = base64.b64decode(encrypted_text)
    nonce = encrypted_text[:16]
    ciphertext = encrypted_text[16:]
    cipher = AES.new(key, AES.MODE_EAX, nonce=nonce)
    plaintext = cipher.decrypt(ciphertext)
    return plaintext.decode('utf-8')


key = "ais3_2025_cycraft"
encrypted_text = "OpR/RJfaK7psxkK9LJPyqT/XbCLOAISR7ryGdSN4kjEStWwyCztV+Rnq0mlx4JbcZiAVdOwEfac="
decrypted_text = decrypt(encrypted_text, key)
print("Decrypted Text:", decrypted_text)

Decrypted Text: rqSYiiBDnOAzT2UzQ4iXRALW22zJvlgImu8sGZg2


In [5]:
# 請註冊一個 ABuseIPDB 的 API Key
# https://www.abuseipdb.com/account
# 然後填在下面 API_KEY

import requests
import random

API_KEY = '815ee3c7d30732ca4a5fdb56986cf17c1b45f0a17df4f2335dffdfc04f381bd29001f6411a34d62d'

url = 'https://api.abuseipdb.com/api/v2/blacklist'
headers = {
    'Key': API_KEY,
    'Accept': 'application/json'
}
params = {
    'confidenceMinimum': 90,
    'limit': 100
}

response = requests.get(url, headers=headers, params=params)
data = response.json()

# 隨機取 2 筆 IP
ips = data['data']
random_ips = random.sample(ips, 2)

for ip in random_ips:
    print(ip['ipAddress'])


181.188.176.247
8.222.142.216


In [8]:
from pymisp import MISPEvent,MISPOrganisation, PyMISP
from dateutil.parser import parser
import json
from pymisp.tools import feed_meta_generator
from io import BytesIO

In [9]:
make_feed = False
misp = None

In [10]:
if make_feed:
    org = MISPOrganisation()
    org.name = "ais3.sectools.tw"
    org.uuid = "d2ded987-c2f1-4b12-bf1d-ba9872d0d452"
else:
    url = "https://misp.sectools.tw"
    key = decrypted_text
    misp = PyMISP(url,key,ssl=False)

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [13]:
from datetime import datetime
from pathlib import Path
blacklist_ip = ["8.222.142.216"]
nick_name = "tea"
for p in blacklist_ip:
    event = MISPEvent()
    event.info = f"[{str(p)}] is suspected of DDoS, found by {str(nick_name)}"
    event.date = datetime.now()
    event.distribution = 3
    event.add_tag('tlp:white')
    if make_feed:
        event.org = org
    else:
        e = misp.search(eventinfo=event.info, metadata=True, pythonify=True)
        if e:
            # Already added.
            continue

    obj_ip = event.add_object(name='domain-ip', standalone=False)
    obj_ip.add_attribute('ip', p)
    obj_ip.add_attribute('first-seen', datetime.now() )
    obj_ip.add_reference(event.uuid, 'related-to',"ais3_friend")

    if make_feed:
        with (Path('output') / f'{event.uuid}.json').open('w') as _w:
            json.dump(event.to_feed(), _w)
    else:
        misp.add_event(event)
    print(event.to_json(indent=2))
if make_feed:
    feed_meta_generator(Path('output'))

/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'misp.sectools.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


{
  "uuid": "daf239ac-a893-4e5a-94be-0ba5b7fb3b8d",
  "Object": [
    {
      "name": "domain-ip",
      "meta-category": "network",
      "template_uuid": "43b3b146-77eb-4931-b4cc-b66c60f28734",
      "description": "A domain/hostname and IP address seen as a tuple in a specific time frame.",
      "template_version": "11",
      "uuid": "ac1ea356-f516-4972-be4d-81f0e8cc173d",
      "ObjectReference": [
        {
          "uuid": "fa8aaf70-4ded-4f78-b532-82b4b6d073d2",
          "object_uuid": "ac1ea356-f516-4972-be4d-81f0e8cc173d",
          "referenced_uuid": "daf239ac-a893-4e5a-94be-0ba5b7fb3b8d",
          "relationship_type": "related-to",
          "comment": "ais3_friend"
        }
      ],
      "Attribute": [
        {
          "uuid": "7b4262ec-1eda-47fc-a0e5-d8e026896264",
          "object_relation": "ip",
          "value": "8.222.142.216",
          "type": "ip-dst",
          "category": "Network activity",
          "disable_correlation": false,
          "to_ids": t